In [27]:
import itertools
import time
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from tqdm import tnrange, tqdm_notebook

%matplotlib inline
plt.style.use('ggplot')

In [30]:
import SelectVariable as sv

In [3]:
dat = pd.read_csv("../data/Cleaned_Data.csv")
dat.head(5)

,ID,Airport,Airline,Age,Gender,Nationality,TripPurpose,TripDuration,FlyingCompanion,ProvinceResidence,...,FrequentDestination3,FrequentDestination4,FrequentDestination5,FrequentDestination6,FrequentDestination7,MileageAirline1,MileageAirline2,MileageAirline3,MileageAirline4,MileageAirline5
0,1,1,1,49,1,1,2,7,0,3,...,0,0,0,0,0,1,0,0,0,0
1,2,1,1,49,2,1,1,4,4,3,...,0,0,0,0,0,0,0,0,0,1
2,3,1,1,25,1,1,1,10,2,3,...,0,0,0,0,0,0,0,0,0,1
3,4,1,1,29,1,1,2,7,2,3,...,0,0,1,0,0,0,1,0,0,0
4,5,1,1,34,2,1,2,4,0,3,...,0,0,0,0,0,0,0,0,0,1


In [5]:
dat = dat.drop(columns =["ID","FrequentDestination1","FrequentDestination2","FrequentDestination3","FrequentDestination4","FrequentDestination5","FrequentDestination6","FrequentDestination7"], axis =1)


In [6]:
dat[["FrequentFlightDestination1","FrequentFlightDestination2","FrequentFlightDestination3","FrequentFlightDestination4","FrequentFlightDestination5","FrequentFlightDestination6","FrequentFlightDestination7","Airport","Airline","Gender","Nationality","TripPurpose","ProvinceResidence","GroupTravel","Destination","DepartureTime","SeatClass","ModeTransport","Occupation","Income","MileageAirline"
         ]]=dat[["FrequentFlightDestination1","FrequentFlightDestination2","FrequentFlightDestination3","FrequentFlightDestination4","FrequentFlightDestination5","FrequentFlightDestination6","FrequentFlightDestination7","Airport","Airline","Gender","Nationality","TripPurpose","ProvinceResidence","GroupTravel","Destination","DepartureTime","SeatClass","ModeTransport","Occupation","Income","MileageAirline"
         ]].astype('category')

In [7]:
dat.head(5)

,Airport,Airline,Age,Gender,Nationality,TripPurpose,TripDuration,FlyingCompanion,ProvinceResidence,GroupTravel,...,AccessCost,AccessTime,Occupation,Income,MileageAirline,MileageAirline1,MileageAirline2,MileageAirline3,MileageAirline4,MileageAirline5
0,1,1,49,1,1,2,7,0,3,2,...,8000.0,40,1,5,1,1,0,0,0,0
1,1,1,49,2,1,1,4,4,3,2,...,8000.0,50,9,3,5,0,0,0,0,1
2,1,1,25,1,1,1,10,2,3,2,...,1000.0,20,12,3,5,0,0,0,0,1
3,1,1,29,1,1,2,7,2,3,2,...,12642.0,76,8,7,2,0,1,0,0,0
4,1,1,34,2,1,2,4,0,3,2,...,8000.0,50,1,3,5,0,0,0,0,1


In [8]:
dat.head(5)

,Airport,Airline,Age,Gender,Nationality,TripPurpose,TripDuration,FlyingCompanion,ProvinceResidence,GroupTravel,...,AccessCost,AccessTime,Occupation,Income,MileageAirline,MileageAirline1,MileageAirline2,MileageAirline3,MileageAirline4,MileageAirline5
0,1,1,49,1,1,2,7,0,3,2,...,8000.0,40,1,5,1,1,0,0,0,0
1,1,1,49,2,1,1,4,4,3,2,...,8000.0,50,9,3,5,0,0,0,0,1
2,1,1,25,1,1,1,10,2,3,2,...,1000.0,20,12,3,5,0,0,0,0,1
3,1,1,29,1,1,2,7,2,3,2,...,12642.0,76,8,7,2,0,1,0,0,0
4,1,1,34,2,1,2,4,0,3,2,...,8000.0,50,1,3,5,0,0,0,0,1


def fit_linear_reg(X,Y):
    #Fit linear regression model and return RSS and R squared values
    model_k = linear_model.LinearRegression(fit_intercept = True)
    model_k.fit(X,Y)
    RSS = mean_squared_error(Y,model_k.predict(X)) * len(Y)
    R_squared = model_k.score(X,Y)
    return RSS, R_squared

#Importing tqdm for the progress bar
from tqdm import tnrange, tqdm_notebook

#Initialization variables
Y = dat["Airport"]
X = dat.drop(columns = 'Airport', axis = 1)
#k = 11
RSS_list, R_squared_list, feature_list = [],[], []
numb_features = []

#Looping over k = 1 to k snip= 11 features in X
for k in tnrange(15,len(X.columns) + 1, desc = 'Loop...'):

    #Looping over all possible combinations: from 11 choose k
    for combo in itertools.combinations(X.columns,k):
        tmp_result = fit_linear_reg(X[list(combo)],Y)   #Store temp result 
        RSS_list.append(tmp_result[0])                  #Append lists
        R_squared_list.append(tmp_result[1])
        feature_list.append(combo)
        numb_features.append(len(combo))   

#Store in DataFrame
df = pd.DataFrame({'numb_features': numb_features,'RSS': RSS_list, 'R_squared':R_squared_list,'features':feature_list})

In [9]:
dat.describe()

,Age,TripDuration,FlyingCompanion,NoTripsLastYear,TotalDepartureHr,Airfare,NoTransport,AccessCost,AccessTime,MileageAirline1,MileageAirline2,MileageAirline3,MileageAirline4,MileageAirline5
count,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.00000,463.000000
mean,39.773218,25.989201,2.781857,3.302376,16.362203,48.717797,1.323974,11067.991361,62.498920,0.161987,0.066955,0.030238,0.25270,0.488121
std,13.462726,71.328230,3.856452,9.195998,3.893605,23.555250,0.541370,19481.481081,57.715149,0.368837,0.250214,0.171426,0.43503,0.500400
min,17.000000,0.000000,0.000000,0.000000,1.000000,3.000000,1.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,29.000000,4.000000,1.000000,1.000000,14.250000,36.000000,1.000000,4000.000000,30.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,37.000000,5.000000,2.000000,2.000000,16.920000,45.210000,1.000000,7000.000000,50.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,50.000000,8.000000,3.000000,3.000000,19.750000,55.250000,2.000000,12642.000000,64.000000,0.000000,0.000000,0.000000,1.00000,1.000000
max,80.000000,730.000000,34.000000,122.000000,25.020000,260.000000,4.000000,350000.000000,390.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [10]:
#Y = dat["Airport"]
#Y =Y.replace(regex={1:0, 2:1})
#X = dat.drop(columns = 'Airport', axis = 1)


In [11]:
def fit_log_reg(X,Y, multi):
    #Fit linear regression model and return RSS and R squared values
    X = sm.add_constant(X)
    model_k = sm.Logit(Y, X.astype(float)).fit()
    if(multi == True):
        model_k=sm.MNLogit(Y, X.astype(float)).fit()    
    AIC = model_k.aic
    BIC = model_k.bic
    LLH = model_k.llf
    R2 = model_k.prsquared
    return AIC, BIC, LLH,R2


In [12]:
#funcion of tring all combination to find the best model
def try_all(dat,  Yname, multi):
    #Initialization variables
    Y = dat[Yname]#dat["Airport"]
    if(Yname =="Airport"):
        Y = Y.replace(regex={1:0, 2:1})
    X = dat.drop(columns = Yname, axis = 1)
    #k = 11
    AIC_list, BIC_list,Log_list,RSquare_list, feature_list = [],[], [],[],[]
    numb_features = []

    #Looping over k = 1 to k snip= 11 features in X
    for k in tnrange(1,len(X.columns) + 1, desc = 'Loop...'):
        #Looping over all possible combinations: from 11 choose k
        for combo in itertools.combinations(X.columns,k):
            tmp_result = fit_log_reg(X.loc[:,list(combo)],Y,multi)   #Store temp result 
            AIC_list.append(tmp_result[0])                  #Append lists
            BIC_list.append(tmp_result[1])
            Log_list.append(tmp_result[2])
            RSquare_list.append(tmp_result[3])
            feature_list.append(combo)
            numb_features.append(len(combo))  
    
    #Store in DataFrame
    df = pd.DataFrame({'numb_features': numb_features,'AIC': AIC_list, 'BIC':BIC_list,'Loglikelihood':Log_list,'McFaddens R2':RSquare_list,'features':feature_list})
    return(df)

In [14]:
#X.loc[:,('Age', 'ProvinceResidence', 'FrequentFlightDestination1')]
#Y = dat["Airline"]#dat["Airport"]
#Y = Y.replace(regex={1:0, 2:1})
#X = dat.drop(columns = "Airline", axis = 1)
#X = sm.add_constant(X)
#model_k = sm.MNLogit(Y, X.astype(float)).fit()
#model_k.summary()
#model_k.llf

In [15]:
dat
resutl = sv.try_all


Optimization terminated successfully.
         Current function value: 0.687179
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684605
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692433
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690497
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683483
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692288
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691556
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.676856
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.686631
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.685214
  

Optimization terminated successfully.
         Current function value: 0.679374
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.675733
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.686804
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.678335
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.677401
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.658204
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.671568
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.672495
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.660543
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.658517
  

In [19]:
resutl.nsmallest(3, ['AIC', 'BIC'])

,numb_features,AIC,BIC,Loglikelihood,McFaddens R2,features
105,5,620.987475,645.813837,-304.493737,0.051126,"(Airline, Age, Nationality, TripPurpose, Flyin..."
71,4,621.284920,641.973555,-305.642460,0.047547,"(Airline, Age, TripPurpose, FlyingCompanion)"
120,6,621.362801,650.326890,-303.681400,0.053658,"(Airline, Age, Gender, Nationality, TripPurpos..."


In [18]:
resutl.nlargest(3, ['Loglikelihood', 'McFaddens R2'])

,numb_features,AIC,BIC,Loglikelihood,McFaddens R2,features
126,7,623.273940,656.375756,-303.636970,0.053796,"(Airline, Age, Gender, Nationality, TripPurpos..."
120,6,621.362801,650.326890,-303.681400,0.053658,"(Airline, Age, Gender, Nationality, TripPurpos..."
123,6,622.915779,651.879868,-304.457889,0.051238,"(Airline, Age, Nationality, TripPurpose, TripD..."


In [20]:
resutl.iloc[105]["features"]

('Airline', 'Age', 'Nationality', 'TripPurpose', 'FlyingCompanion')

In [21]:
#X.loc[:,('Age', 'ProvinceResidence', 'FrequentFlightDestination1')]
Y = dat["Airport"]#dat["Airport"]
Y = Y.replace(regex={1:0, 2:1})
X = dat.drop(columns = "Airport", axis = 1)
X = sm.add_constant(X)
model_k = sm.Logit(Y, X.astype(float)).fit()
model_k.summary()
#model_k.llf

Optimization terminated successfully.
         Current function value: 0.282354
         Iterations 31


/opt/conda/lib/python3.6/site-packages/statsmodels/base/model.py:1092: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Airport   No. Observations:                  463
Model:                          Logit   Df Residuals:                      430
Method:                           MLE   Df Model:                           32
Date:                Fri, 24 May 2019   Pseudo R-squ.:                  0.5926
Time:                        21:21:47   Log-Likelihood:                -130.73
converged:                       True   LL-Null:                       -320.90
                                        LLR p-value:                 3.283e-61
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          3.6608        nan        nan        nan         nan         nan
Airline                        0.5706      0.153      3.723      0.000       0.270       0.871
Age                            0.0279      0.013      2.101      0.036       0.002       0.054
Gender                         0.3922      0.359      1.092      0.275      -0.311       1.096
Nationality                    0.5573      0.188      2.972      0.003       0.190       0.925
TripPurpose                    0.3356      0.227      1.479      0.139      -0.109       0.780
TripDuration                   0.0013      0.002      0.553      0.581      -0.003       0.006
FlyingCompanion                0.0947      0.051      1.840      0.066      -0.006       0.196
ProvinceResidence              0.1435      0.080      1.799      0.072      -0.013       0.300
GroupTravel                    0.2333      0.555      0.421      0.674      -0.854       1.320
NoTripsLastYear               -0.0529      0.046     -1.161      0.246      -0.142       0.036
FrequentFlightDestination1    -3.5171      0.741     -4.749      0.000      -4.969      -2.066
FrequentFlightDestination2    -1.6797      0.718     -2.339      0.019      -3.087      -0.272
FrequentFlightDestination3    -0.3581      0.678     -0.528      0.597      -1.686       0.970
FrequentFlightDestination4    -2.3841      0.900     -2.649      0.008      -4.148      -0.620
FrequentFlightDestination5    -3.0881      0.901     -3.427      0.001      -4.854      -1.322
FrequentFlightDestination6    -2.7565      0.921     -2.991      0.003      -4.563      -0.950
FrequentFlightDestination7    -1.9635      1.285     -1.528      0.127      -4.483       0.556
Destination                   -1.2495      0.255     -4.894      0.000      -1.750      -0.749
TotalDepartureHr               0.1877      0.069      2.734      0.006       0.053       0.322
DepartureTime                 -2.7320      0.412     -6.634      0.000      -3.539      -1.925
SeatClass                     -0.1607      0.384     -0.419      0.675      -0.913       0.592
Airfare                       -0.0100      0.007     -1.413      0.158      -0.024       0.004
NoTransport                   -1.0147      0.409     -2.479      0.013      -1.817      -0.212
ModeTransport                 -0.1522      0.080     -1.892      0.058      -0.310       0.005
AccessCost                   1.51e-05   6.88e-06      2.194      0.028    1.61e-06    2.86e-05
AccessTime                    -0.0076      0.004     -2.063      0.039      -0.015      -0.000
Occupation                     0.0050      0.050      0.100      0.920      -0.093       0.103
Income                        -0.1175      0.128     -0.916      0.360      -0.369       0.134
MileageAirline                 0.6671        nan        nan        nan         nan         nan
MileageAirline1                2.5664        nan        nan        nan         nan         nan
MileageAirl